install pytorch-geometric
on mac: using pip wheel installation instructions, with python 3.9 and pytorch via conda (https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html)

loading data: 
ultimately: build on-disk dataset w/ pytorch 
atm: build dataset on the fly with synthetic dataset + data loader

In [1]:
import numpy as np
# coords used in segmentation paper (polar is 2d, how?)
#relative_polar = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/relative_coords_polar_3.npy')
#relative_coord = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/relative_coords_3.npy')
#edge_index_seg = np.load('/Users/hannah.spitzer/projects/MELD/Benchmarking-Surface-DL/Segmentation_UGSCNN/GraphMethods/data/edge_index_3.npy')

In [2]:
import importlib
import meld_graph
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
importlib.reload(meld_graph)
importlib.reload(meld_graph.models)
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.experiment)

import logging
logging.basicConfig(level=logging.INFO)

Setting MELD_DATA_PATH to /home/co-spit1/meld_data
Setting BASE_PATH to /home/co-spit1/meld_data
Setting EXPERIMENT_PATH to /home/co-spit1/meld_experiments
Setting FS_SUBJECTS_PATH to /home/co-spit1/meld_data/output/fs_outputs
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1


In [3]:
network_parameters = {
    'network_type': 'MoNet',
    'model_parameters': {
        'layer_sizes': [16,10],
        'dim': 2, # pseudo-coord dim
    },
    'training_parameters': {
        "max_patience": 10,
        "num_epochs": 100,
        'lr': 1e-4,
        'loss': 'cross_entropy',
        "batch_size": 1,
        "shuffle_each_epoch": True,
    }
}

data_parameters = {
    'hdf5_file_root': "{site_code}_{group}_featurematrix.hdf5",
    'site_codes': ['H4'],
    'scanners': ['15T', '3T'],
    'dataset': 'MELD_dataset_V6.csv',
    'group': 'both',
    "features_to_exclude": [],
    "subject_features_to_exclude": [],
    "features": ['.on_lh.curv.mgh',
        '.on_lh.gm_FLAIR_0.25.mgh',
        '.on_lh.gm_FLAIR_0.5.mgh',
        '.on_lh.gm_FLAIR_0.75.mgh',
        '.on_lh.gm_FLAIR_0.mgh',
        '.on_lh.pial.K_filtered.sm20.mgh',
        '.on_lh.sulc.mgh',
        '.on_lh.thickness.mgh',
        '.on_lh.w-g.pct.mgh',
        '.on_lh.wm_FLAIR_0.5.mgh',
        '.on_lh.wm_FLAIR_1.mgh'],
    "features_to_replace_with_0": [], # specify this if manually specifying features
    "number_of_folds": 10,
    "fold_n": 0,
    "preprocessing_parameters": {
        "scaling": "scaling_params_GDL.json"
    },
    "combine_hemis": "stack",  # "currently just stack is implemented"
}



In [4]:
exp = meld_graph.experiment.Experiment(network_parameters, data_parameters, save=False)
_ = exp.get_train_val_test_ids()
exp.data_parameters['train_ids'] = exp.data_parameters['train_ids'][:2]
exp.data_parameters['val_ids'] = exp.data_parameters['val_ids'][:2]
ds = meld_graph.dataset.GraphDataset.from_experiment(exp, mode='train')

INFO:meld_graph.experiment:getting train val test split
INFO:meld_classifier.meld_cohort:total number of subjects: 88
INFO:meld_classifier.meld_cohort:total number of subjects after restricting to subjects from MELD_dataset_V6.csv: 86
INFO:meld_classifier.meld_cohort:total number of subjects: 1158
INFO:meld_classifier.meld_cohort:total number of subjects after restricting to subjects from MELD_dataset_V6.csv: 945
INFO:meld_classifier.meld_cohort:total number after filtering by scanner ['3T', '15T'], features, lesional_only True: 911
INFO:meld_classifier.meld_cohort:full_feature_list: ['.inter_z.asym.on_lh.curv.mgh', '.inter_z.asym.on_lh.intra_z.gm_FLAIR_0.25.sm10.mgh', '.inter_z.asym.on_lh.intra_z.gm_FLAIR_0.5.sm10.mgh', '.inter_z.asym.on_lh.intra_z.gm_FLAIR_0.75.sm10.mgh', '.inter_z.asym.on_lh.intra_z.gm_FLAIR_0.sm10.mgh', '.inter_z.asym.on_lh.intra_z.pial.K_filtered.sm20.mgh', '.inter_z.asym.on_lh.intra_z.thickness.sm10.mgh', '.inter_z.asym.on_lh.intra_z.w-g.pct.sm10.mgh', '.inter_z.

In [ ]:
exp.train()

/home/co-spit1/.conda/envs/meld_graph/lib/python3.9/site-packages/torch/cuda/__init__.py:143: UserWarning: 
NVIDIA A100-SXM-80GB with CUDA capability sm_80 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA A100-SXM-80GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [11]:
from meld_classifier.meld_cohort import MeldCohort

cohort = MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat6.hdf5', )

Init signature:
MeldCohort(
    hdf5_file_root='{site_code}_{group}_featurematrix_combat.hdf5',
    dataset=None,
    data_dir='',
)
Docstring:      Class to define cohort-level parameters such as subject ids, mesh
File:           ~/projects/MELD/meld_classifier/meld_classifier/meld_cohort.py
Type:           type
Subclasses:     
